In [ ]:
import numpy as np
import emcee
import scipy.stats as stats
import matplotlib.pyplot as plt

offset = 0
days = np.array([
    0.    ,      2.9981597  , 5.9844097  , 8.9885417,  10.0159028 , 12.0850347,
 13.0630556,  15.0476968 , 16.0428357 , 17.925463  , 18.9360648 , 20.9814005,
 23.94114034, 26.9181366 , 31.96125377, 34.9793519 , 35.9669213 , 37.9624885,
 41.0245255,  43.0386459 , 43.9345255 , 46.9047222 , 48.9534028  ,49.9327778,
 52.921169 ,  56.9240047 , 58.9151968 , 61.9748264  ,62.9192246 , 74.8983797,
 77.88153653 ,84.8774769 ])[offset:]
obs = np.array([
    0.0434924 , 0.08507323 ,0.14708396, 0.176258 ,  0.1886173  ,0.20833433,
 0.18773103 ,0.23081693 ,0.18828987 ,0.19365838 ,0.17786331, 0.17048814,
 0.1378152  ,0.11589608 ,0.10049781, 0.09370225, 0.08094249, 0.09205468,
 0.07899659 ,0.07067109, 0.07630035 ,0.06387476, 0.05980897 ,0.06007613,
 0.04403494 ,0.04053279, 0.0445369 , 0.03919224, 0.03171029, 0.03399923,
 0.02664128 ,0.02762099])[offset:]
obse = np.array([
    0.00549603, 0.00797799, 0.01049272 ,0.01437629, 0.01122938 ,0.01705866,
 0.01256814 ,0.0276811 , 0.01001263 ,0.0137053 , 0.01546216, 0.01401168,
 0.00866173, 0.00997611, 0.00921919, 0.01192447, 0.0088945 , 0.01091706,
 0.01150647, 0.00905011, 0.00836018, 0.00711094, 0.00596107 ,0.00694383,
 0.00696718, 0.00698428, 0.0082303,  0.01012755 ,0.00651753 ,0.0058564,
 0.00606112 ,0.00482436])[offset:]

theta = [0.3261999562177893, -0.6950993653056392, 27.457500531767177, 0.6220350943386425, 4.4586949870056305, 31.857462881453372]

In [ ]:


EPS = 1e-10

def sgm(x, x0, s):
    return 1/(1 + np.exp(-s*(x-x0)))

def syn_sne_sfunc(t, A, t0, gamma, f, trise, tfall,
    #s=1/3,
    ):
    #s = 1/3,
    s = 1/5
    g = sgm(t, gamma+t0, s)
    early = 1.0*(A*(1 - (f*(t-t0)/gamma))   /   (1 + np.exp(-(t-t0)/trise)))
    late = 1.0*(A*(1-f)*np.exp(-(t-(gamma+t0))/tfall)   /   (1 + np.exp(-(t-t0)/trise)))
    flux = (1-g)*early + g*late
    return flux

def log_prior(theta):
    A, t0, gamma, f, trise, tfall = theta
    lp_A = np.log(stats.gamma.pdf(A, 1.5614117006329225, 0, 1/3.596688103142064)+EPS)
    lp_t0 = np.log(stats.norm.pdf(t0, 4.68021259907846, 5.359761365339392)+EPS)
    lp_gamma = np.log(stats.gamma.pdf(gamma, 3.7788735568722034, 1, 1/0.15241503523503291)+EPS)
    lp_f = 0 if f>0 and f<1 else -np.infty
    lp_trise = np.log(stats.gamma.pdf(trise, 2.8972409516753013, 1, 1/1.077805262206754)+EPS)
    lp_tfall = np.log(stats.gamma.pdf(tfall, 1.6747508057839176, 1, 1/0.033776715649123344)+EPS)
    return lp_A + lp_t0 + lp_gamma + lp_f + lp_trise + lp_tfall

def log_likelihood(theta, days, obs, obse):
    model = syn_sne_sfunc(days, *theta)
    sigma2 = obse**2
    return -0.5 * np.sum((obs - model)**2/sigma2 + np.log(sigma2))

def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

n_trace_samples = 480
n_tune = 1000
n_chains = 6*2

assert n_trace_samples%nwalkers==0
pos = np.array(theta) + 1e-4 * np.random.randn(nwalkers, 6)
sampler = emcee.EnsembleSampler(nwalkers, theta0.shape[-1], log_probability, args=(days, obs, obse))
progress = 0
sampler.run_mcmc(pos, (n_trace_samples+n_tune)//nwalkers, progress=progress)
pass

In [ ]:
trace = sampler.get_chain(discard=n_tune//nwalkers, flat=True)
trace.shape